# Start

Dieses Skript führt Sie Schritt für Schritt durch die Programmierung Neuronaler Netze in Python/Tensorflow. 

Über den Pfeil neben der Abschnittsnummer können die einzelnen Code-Abschnitte ein- und ausgeklappt werden. Manche Kapitel beinhalten weitere Unterabschnitte.
Über den "Ausführen"/ "Execute Cell"-Button links neben einem Code-Block wird der entsprechende Code-Abschnitt ausgeführt. Das Programm-Feedback (prints) werden unter dem zugehörigen Code-Abschnitt angezeigt. 

Nur in Visual Studio Code: Nach dem Starten von Visual Studio Code muss zunächst ein Python Interpreter ausgewählt werden. Beim Versuch einen Code-Block auszuführen öffnet sich ein Auswahlfenster, in dem Sie am besten die aktuellste Python-Version auswählen.

Falls Änderungen im Code vorgenommen werden, muss nicht der gesamte Code neu ausgeführt werden, sondern nur von der Änderung abhängige Abschnitte. 

In diesem Tutorial trainieren Sie ein Neuronales Netz darauf , die Handgesten für Schere, Stein und Papier zu erkennen. Der Datensatz enthält 2.892 Bilder von verschiedenen Händen in Stein-Papier-Schere-Posen. Alle Bilder wurden mit Hilfe von CGI-Techniken erzeugt, was den Vorteil bietet, dass keine realen Fotos aufgenommen werden mussten. Jedes Bild ist 300×300 Pixel groß und hat 24-Bit-Farbe.

Am Ende des Skriptes finden sich ein paar Aufgaben, deren Bearbeitung Ihr Verständnis zur Thematik fördern soll.

Große Teile dieses Skriptes basieren auf einem Tutorial von:
https://colab.research.google.com/github/trekhleb/machine-learning-experiments/blob/master/experiments/rock_paper_scissors_cnn/rock_paper_scissors_cnn.ipynb#scrollTo=k2zZOCweLt1-

Der verwendete Datensatz ist hier zu finden: https://www.kaggle.com/datasets/sanikamal/rock-paper-scissors-dataset 

# 0. Imports

Zunächst werden die erforderlichen Bibliotheken importiert.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import platform
import os
import requests
from zipfile import ZipFile

print('Python version:', platform.python_version())
print('Tensorflow version:', tf.__version__)
print('Keras version:', tf.keras.__version__)

# 1. Laden des Datensets


Die TensorFlow Bibliothek stellt einige Datensätze für Tutorials zur Verfuegung, unter anderem den Datensatz mit Bildern der Gesten Schere, Stein, Papier. 
Damit sich der Workflow in diesem Tutorial und die Programmierung am Präsenztermin ähneln, wird der Datensatz aus einem git-reposotory heruntergeladen.

In [ ]:
#Herunterladen des Datensatzes

URL = "https://seafile.projekt.uni-hannover.de/f/0a3bf5ffb2fc44f1b687/?dl=1"
response = requests.get(URL)
open("Rock_Paper_Scissors_Dataset.zip", "wb").write(response.content)

zip = ZipFile('Rock_Paper_Scissors_Dataset.zip')
zip.extractall()

In [ ]:
# Laden des Datensatzes
Directory = os.path.abspath(os.getcwd())+"/Rock-Paper-Scissors_Keras_Dataset"

Split=0.2
Default_Image_Size = (300,300)      # Höhe, Breite
INPUT_IMG_SHAPE=(300,300,3)        # Höhe, Breite, Farbkanäle

dataset_train_raw = tf.keras.utils.image_dataset_from_directory(
    Directory, 
    labels='inferred', 
    label_mode='int',
    class_names=None, 
    color_mode='rgb', 
    batch_size = 32, 
    image_size=Default_Image_Size, 
    shuffle=True, 
    seed=1, 
    validation_split=Split, 
    subset='training',
    interpolation='bilinear', 
    ) 
# bis hier

dataset_test_raw = tf.keras.utils.image_dataset_from_directory(
    Directory, 
    labels='inferred', 
    label_mode='int',
    class_names=None, 
    color_mode='rgb', 
    batch_size = 32, 
    image_size=Default_Image_Size, 
    shuffle=True, 
    seed=1, 
    validation_split=Split, 
    subset='validation',
    interpolation='bilinear', 
    )

class_names = dataset_train_raw.class_names

NUM_TRAIN_EXAMPLES = len(dataset_train_raw.file_paths)
NUM_TEST_EXAMPLES = len(dataset_test_raw.file_paths)
NUM_CLASSES=len(class_names)

print('Label 1: ',class_names[0])
print('Label 2: ',class_names[1])
print('Label 3: ',class_names[2])

dataset_train_raw = dataset_train_raw.unbatch()
dataset_test_raw = dataset_test_raw.unbatch() 


In [ ]:
#os.system("git clone {}".format('https://gitlab.uni-hannover.de/robin-menzel/Programming_MLL/Rock-Paper-Scissors_Keras_Dataset.git')) 

#os.system("git clone {}".format('https://seafile.cloud.uni-hannover.de/d/f56e56a8e77c413e8434/')) 
os.system('https://seafile.cloud.uni-hannover.de/f/a41617b0b991446c8ba4/?dl=1')


#os.system("git clone {}".format('https://github.com/match-PM/Masterlabor-MLL')) 


# 2. Festlegen der Bildgröße und Image-Augmentaion

## 2.1 Festlegen der Bildgröße und Erzeugung eines Datensatzes mit verringerter Bildgröße

Um den Rechenaufwand für das NN zu verringern werden Bilder eines Datensatzes i.d.R. in ihrer Pixel-Größe reduziert. 


In [ ]:
INPUT_IMG_SHAPE_REDUCED = (
    INPUT_IMG_SHAPE[0]//2,
    INPUT_IMG_SHAPE[1]//2,
    INPUT_IMG_SHAPE[2]
)

print('Input image shape (original):', INPUT_IMG_SHAPE)
print('Input image shape (reduced):', INPUT_IMG_SHAPE_REDUCED)

#Diese Funktion formatiert die Bildgroeße eines Bildes von der Urspruenglichen in INPUT_IMG_SIZE
def format_example(image, label):
    # Konvertiert Bildfarbwerte zu float
    image = tf.cast(image, tf.float32)
    # Konvertiert Bildfarbwerte zu Werten zwischen [0,1]
    image = image / 255.
    # Konvertiert Bildgroesse zu [INPUT_IMG_SIZE, INPUT_IMG_SIZE]
    image = tf.image.resize(image, [INPUT_IMG_SHAPE_REDUCED[0], INPUT_IMG_SHAPE_REDUCED[1]])
    return image, label

# Mit der map-Funktion wird eine Funktion auf ein Datenset angewendet 
dataset_train = dataset_train_raw.map(format_example)
dataset_test = dataset_test_raw.map(format_example)


## 2.2 Definition von Image-Augmentation Funktionen und Erzeugung eines Augmented Datensatzes

Durch den Einsatz von Imgage-Augmentaion können die Bilder eines Datensatzes für das Training manipuliert werden. Hierdurch sieht das Netz beim Training immer eine Variation des Originalbildes, wodruch der Effekt von Overfitting reduziert werden kann.

In [ ]:

def augment_color(image):   # Verändert die Sätigung, die Helligkeit, den Kontrast und den Farbton eines Bildes zufällig
    image = tf.image.random_hue(image, max_delta=0.08)
    image = tf.image.random_saturation(image, lower=0.7, upper=1.3)
    image = tf.image.random_brightness(image, 0.05)
    image = tf.image.random_contrast(image, lower=0.8, upper=1)
    image = tf.clip_by_value(image, clip_value_min=0, clip_value_max=1)
    return image

def augment_rotation(image): # Dreht das Bild zufällig in einem Bereich von 0-90°
    # Rotate 0, 90
    return tf.image.rot90(
        image,
        tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
    )

def augment_inversion(image):   # Invertiert ein Bild nach zufälliger Auswahl
    random = tf.random.uniform(shape=[], minval=0, maxval=1)
    if random > 0.5:
        image = tf.math.multiply(image, -1)
        image = tf.math.add(image, 1)
    return image

def augment_zoom(image, min_zoom=0.8, max_zoom=1.0):    # Zoomt ein Bild zufällig heran
    image_width, image_height, image_colors = image.shape
    crop_size = (image_width, image_height)

    # Generiert Beschnitteinstellungen von 1 % bis 20 % 
    scales = list(np.arange(min_zoom, max_zoom, 0.01))
    boxes = np.zeros((len(scales), 4))

    for i, scale in enumerate(scales):
        x1 = y1 = 0.5 - (0.5 * scale)
        x2 = y2 = 0.5 + (0.5 * scale)
        boxes[i] = [x1, y1, x2, y2]

    def random_crop(img):
        # Generiert eine zufällige Beschnitteinstellungen für ein Bild
        crops = tf.image.crop_and_resize(
            [img],
            boxes=boxes,
            box_indices=np.zeros(len(scales)),
            crop_size=crop_size
        )
        # Return zufällige Beschnitteinstellung
        return crops[tf.random.uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]

    choice = tf.random.uniform(shape=[], minval=0., maxval=1., dtype=tf.float32)

    # Zufällig, nur 50% der Fälle
    return tf.cond(choice < 0.5, lambda: image, lambda: random_crop(image))

# Fasst alle Augment-Funktionen in einer Funktion Zusammen
def augment_data(image, label):
    image = augment_zoom(image)
    image = augment_color(image)
    image = augment_rotation(image)
    image = augment_inversion(image)
    return image, label

# Wendet die Augment-Funktion auf das Datenset an

dataset_train_augmented = dataset_train.map(augment_data)

# 3 Vergleiche Datensatz mit reduzierter Größe und Augmented-Datensatz

In [ ]:
# Mit dieser Funktion können die ersten 12 Bilder eines Datzensatzes in einem 3x4 Plot dargestellt werden - wird in Abschnitt 3 genutzt
def preview_dataset(dataset):               
    plt.figure(figsize=(12, 12))
    plot_index = 0
    for features in dataset.take(12):
        (image, label) = features
        plot_index += 1
        plt.subplot(3, 4, plot_index)
        label = class_names[label]
        plt.title('Label:' + label)
        plt.imshow(image.numpy())
        plt.axis("off")

# So sehen die ersten 12 Bilder des dataset_train Datensatzes aus
preview_dataset(dataset_train)

# So sehen die ersten 12 Bilder des dataset_train_augmented Datensatzes aus
preview_dataset(dataset_train_augmented)

# 4. Mischen der Bilder im Datensatz und Unterteilung in Batches


In [ ]:
BATCH_SIZE = 30 # Hier wird die Batchgröße festgelegt    #Default 30

dataset_train_augmented_shuffled = dataset_train_augmented.shuffle(# Shuffled den augmentierteb Trainingsdatensatz und speichert es in einer neuen Datenssatzvariable
    buffer_size=NUM_TRAIN_EXAMPLES,seed=1
)

# Konvertiert den Trainingsdatensatz (augmentiert und geshuffelt) in ein Batched Datensatz
dataset_train_augmented_shuffled = dataset_train_augmented_shuffled.batch(
    batch_size=BATCH_SIZE
)

# Prefetch erlabut der Input Pipeline Datenbaches bereits während des Trainings zu laden
dataset_train_augmented_shuffled = dataset_train_augmented_shuffled.prefetch(
    buffer_size=tf.data.experimental.AUTOTUNE
)

# Batched den Testdatensatz
dataset_test_shuffled = dataset_test.batch(BATCH_SIZE)

# 5. Erstellung eines sequenziellen Modells

Modelle können in Keras als sequenzielle (sequential) und funktionale (functional) Modelle erstellt werden. 
Für Einsteiger sind sequenzielle Modelle einfacher zu implementieren.

In diesem Abschnitt wird ein sequenzielles Modell aufgebaut.

In [ ]:
# Erstellt ein neues Sequenzielles Modell mit dem Variablenname "model"
model = tf.keras.models.Sequential()

# Fügt ein Convolution-Layer (1.Layer) hinzu
model.add(tf.keras.layers.Convolution2D(
    input_shape=INPUT_IMG_SHAPE_REDUCED,
    filters=64,                                 # Anzahl zu trainierender Filter/ Anzahl zu erzeugender Feature Maps
    kernel_size=3,                              # Größe der Faltungsmatrizen
    activation=tf.keras.activations.relu        # Aktivierungsfunktion der trainierten Filter
))

# Fügt ein MaxPooling-Layer hinzu
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),                           # Anzahl an Pixeln für Pooling
    strides=(2, 2)                              # Abstand für das Pooling
))

# Fügt ein Convolution-Layer hinzu
model.add(tf.keras.layers.Convolution2D(
    filters=64,
    kernel_size=3,
    activation=tf.keras.activations.relu
))

# Fügt ein MaxPooling-Layer hinzu
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2)
))

# Fügt ein Convolution-Layer hinzu
model.add(tf.keras.layers.Convolution2D(
    filters=128,
    kernel_size=3,
    activation=tf.keras.activations.relu
))
# Fügt ein MaxPooling-Layer hinzu
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2)
))

# Fügt ein Convolution-Layer hinzu
model.add(tf.keras.layers.Convolution2D(
    filters=128,
    kernel_size=3,
    activation=tf.keras.activations.relu
))

# Fügt ein MaxPooling-Layer hinzu
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2)
))

# Glätten (Flatten) der Netzstruckur für das Dense-Layer
model.add(tf.keras.layers.Flatten())

# Fügt ein Dropout-Layer hinzu
model.add(tf.keras.layers.Dropout(0.5))

# Fügt ein Dense-Layer mit 512 Neuronen und Relu-Aktivierungsfunktion hinzu
model.add(tf.keras.layers.Dense(
    units=512,                              #Anzahl an Neuronen     
    activation=tf.keras.activations.relu
))

# Output-Layer mit Softmax-Aktivierungsfunktion 
model.add(tf.keras.layers.Dense(
    units=NUM_CLASSES,                      #Anzahl an Neuronen
    activation=tf.keras.activations.softmax
))

# Mit diesem Befehl kann die Struktur eines Modelles ausgegeben werden. Am Ende der Tabelle wird angegeben, wie viele Parameter das erzeugte Netz aufweist und wie viele davon trainierbar bzw. festgelegt sind.
# In diesem Fall müssen alle Parameter trainiert werden. 
# Mithilfe der Spalte "Output Shape" kann nachvollzogen werden, welche Auswirkungen das hinzufügen eines Layers auf die Anzahl der Parameter hat. Zur Erinnerung: Unser Input hat Shape (150, 150,3)
model.summary()

# 6. Modell kompilieren

In diesem Abschnitt werden die Einstellungen für das Training des Neuronalen Netzes, wie die zu verwendende Fehlergütefunktional, der Optimierungsalgorithmus und die Lernrate festgelegt.


In [ ]:
# adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)            # Alternativ kann auch der Adam Optimizer genutzt werden
rmsprop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

model.compile(
    optimizer=rmsprop_optimizer,
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
)

steps_per_epoch = NUM_TRAIN_EXAMPLES // BATCH_SIZE
validation_steps = NUM_TEST_EXAMPLES // BATCH_SIZE

# 7. Modell Training

## 7.1 Training

Mit der Funktion model.fit() wird das Training des Neuronalen Netzes auf den Trainingsdatensatz gestartet.
Dies kann mehrere Minuten dauern.

In [ ]:
train_epoches=3 # Festlegen der Trainingsepochen 

training_history = model.fit(
    x=dataset_train_augmented_shuffled.repeat(),        # Festlegen des Trainingsdatensets
    validation_data=dataset_test_shuffled.repeat(),     # Festlegen des Validationdatensets
    epochs=train_epoches,
    steps_per_epoch=steps_per_epoch,                    # Muss nur definiert werden, wenn dataset.repeat() in fit() verwendet wird, damit der Trainingsalgorithmus eine Stop Condition hat.
    validation_steps=validation_steps,                  # Muss nur definiert werden, wenn dataset.repeat() in fit() verwendet wird
    verbose=1)

## 7.2  Trainingshistorie grafisch abbilden

In [ ]:
# Diese Funktion stellt die Trainings-Historie grafisch dar
def render_training_history(training_history):
    loss = training_history.history['loss']
    val_loss = training_history.history['val_loss']

    accuracy = training_history.history['accuracy']
    val_accuracy = training_history.history['val_accuracy']

    plt.figure(figsize=(14, 4))

    plt.subplot(1, 2, 1)
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.plot(loss, label='Training set')
    plt.plot(val_loss, label='Test set', linestyle='--')
    plt.legend()
    plt.grid(linestyle='--', linewidth=1, alpha=0.5)

    plt.subplot(1, 2, 2)
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.plot(accuracy, label='Training set')
    plt.plot(val_accuracy, label='Test set', linestyle='--')
    plt.legend()
    plt.grid(linestyle='--', linewidth=1, alpha=0.5)

    plt.show()

# Grafische Darstellung der Trainings-Historie
render_training_history(training_history)

# 8. Speichern und Laden von Modellen

So kann ein trainiertes NN gespeichert und geladen werden. Dies ist z.B. sinnvoll falls das trainierte Netz in einer Anwendung genutzt werden soll. Z.B. können Sie durch das Laden Ihres Netzes in einer späteren Session die Live-Kameravorhersage erneut dürchführen, ohne erneut ein Netz trainieren zu müssen.

In [ ]:
model_name = 'My_Model_R_P_S.h5'
# Modell wird unter dem Namen "model_name" gespeichert
model.save(model_name, save_format='h5')

In [ ]:
# Läd das Modell mit dem Namen "model_name" und weist es der Variablen "model" zu
model = tf.keras.models.load_model('My_Model_R_P_S.h5')

# 9. Klassifizierungsvorhersagen aus Bildern des Test_Datensatzes

Mit der Funktion predict() kann das NN für Vorhersagen genutzt werden. 

In diesem Abschnitt  werden für die ersten 12 Bilder des 1. Batches des Testdatensatzes vorhersagen gemacht. 

Die Bilder, sowie die Klassenvorhersage und das tatsächliche Label werden dargestellt. 

In [ ]:
for images, label in dataset_test_shuffled.take(3): # Take 1 Batch, Bei 3 -> Take 3 Batches
  prediction=model.predict(images)                  # Matrix des Größe (Batch_Size x Anz. der Klassen) gefüllt mit Konfidenzwerten 
  plt.figure(figsize=(15,10))

  for i in range(12):                               # Ersten 12 Bilder eines Batches
    predicted_class=np.argmax(prediction[i])        # Welche Klasse hat den höchsten Konfidenzwert
    plt.subplot(3, 4, i + 1)
    plt.axis("off")
    plt.title('P: '+ class_names[predicted_class]+ " L: "+class_names[label[i]], fontsize=15)  # P = Prediction, L = Label from Picture
    plt.imshow(images[i])
    plt.axis("off")

# 10. Vorhersagen aus Live-Kamerabild

Das NN kann auch genutzt werden um Live-Vorhersagen aus dem Kamerabild zu machen. 
Damit dieser Codeabschnitt funktioniert brauchen Sie entwieder eine integrierte Laptopkamera oder besser, eine externe Webcam.

Nach dem Ausführen wird ein Kamerabild, mit dem vorhergesagten Label und dem Konfidenzwert angezeigt.
Mit der Taste 'q' wird das Kamerabild geschlossen.

In manchen Fällen hat dieser Teil in der Vergangenheit nicht auf den persönlichen Computern der Teilnehmer funktioniert. Falls dies bei Ihnen der Fall sein sollte, können Sie diesen Teil auch über das Python File "Predict with Camera.py" aufrufen. Achten Sie darauf, dass sich der Code in demselben Ordner befinden muss, wie das gespeicherte Model "My_Model_R_P_S.h5".

Sollte das Python File ebenfalls nicht funktionieren, wenden Sie sich an den Betreuer der Veranstaltung. 

In [ ]:
import cv2 

IMAGE_HIGHT=INPUT_IMG_SHAPE_REDUCED[0]
IMAGE_WIDTH=INPUT_IMG_SHAPE_REDUCED[1]

Threshold = 0.7

def FormatingDataset(images):
  images = tf.cast(images, tf.float32)
  images = images / 255.
  images = tf.image.resize(images, (IMAGE_HIGHT, IMAGE_WIDTH))
  return images

VideoFeed = cv2.VideoCapture(0)                        # Falls kein VideoFeed angezeigt wird, die 0 durch 1, 2, etc. ersetzen
width = int(VideoFeed.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(VideoFeed.get(cv2.CAP_PROP_FRAME_HEIGHT))

while VideoFeed.isOpened(): 
    ret, frame = VideoFeed.read()
    images = FormatingDataset(frame)
    img_pred=(np.expand_dims(images,axis=0))

    prediction=model.predict(img_pred)
    predicted_class=np.argmax(prediction)
    if (prediction[0][predicted_class] > Threshold):
        text=str(class_names[predicted_class] + "  Value: " + str(prediction[0][predicted_class]))
        cv2.putText(frame,text,(int(0),int(50)),cv2.FONT_HERSHEY_TRIPLEX, 0.8, (0,255,0),1)
    cv2.imshow('Stream',frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        VideoFeed.release()
        cv2.destroyAllWindows()
        break

# 11. Aufgaben

Wichtiger Hinweis für die Aufgaben: Nach Änderungen an Datensätzen und Trainingsparameter muss das Modell (Abschnitt 5 + Abschnitt 6) neu geladen werden. Nach dem Training werden die Parameter eines Modells gespeichert, sodass bei erneutem Training das Optimierungsproblem einen anderen Ausgangspunkt hat. Z.B. kann das Modell bereits bestmöglich angepasst sein, wodurch sich die Werte in der Trainingshistorie evtl. nicht mehr sichtbar verbessern.

1. Variieren Sie die Batch_Size, die Epochenzahl und die Lernrate wie unten angeben und trainieren Sie das Netz. Betrachten Sie die Trainingshistorie sowie die Rechenzeit. Was können Sie aus den Versuchen für den Einfluss dieser Parameter ableiten? Hinweis: Die Rechenzeit wird unten links im Code-Block angezeigt. Initialisieren Sie das Model nach jeden Schritt neu (Abschnitt 5 + Abschnitt 6).

    Batch-Size = 5 -              Epochen = 15 -         Lernrate = 0.001

    Batch-Size = 30 -             Epochen = 15 -         Lernrate = 0.001

    Batch-Size = 100 -            Epochen = 15  -        Lernrate = 0.001

    Die Plots der Trainingshistorien der folgenden Versuche wurden Ihnen bei Stud.IP bereitgestellt und müssen deshalb nicht von Ihnen erzeugt werden. 

    Batch-Size = 30 -             Epochen = 15 -         Lernrate = 0.01

    Batch-Size = 30 -             Epochen = 5 -          Lernrate = 0.001

    Batch-Size = 30 -             Epochen = 50 -         Lernrate = 0.001
    

Setzen Sie die Parameter für die folgenden Aufgaben auf Batch-Size=30, Epochen=15 und Lernrate=0.001.

2. Vergleiche welchen Einfluss die Image-Augmentation auf das Trainingsergebnis hat. Trainiere das Netz mit und ohne Image Augmentation und Vergleiche Validation_Loss und Validation_accuracy (der Graf mit Image-Augmantaion kann auch aus Aufgabe 1 entnommen werden). Stellen Sie dafür die grafischen Darstellungen der Trainingshistorie gegenüber. Was leiten Sie als Nutzen der Image Augmentation ab?

3. Überlegen Sie, welche Image-Augmentation Funktionen sinnvoll sind und welche nicht? Fügen Sie eine Funktion zur Image-Augmentation hinzu, welche die Bilder des Datensatzes zufällig entlang der horizontalen und vertikalen Achse spiegelt.
Dies ist mit den folgenden Funktionen möglich:

    image = tf.image.random_flip_left_right(image)
    
    image = tf.image.random_flip_up_down(image)

Trainieren Sie Ihr Netz anschließend mit Batch-Size=30, Epochen=20 und Lernrate=0.001.

4. Vergleiche, wie die Trainingshistorie mit einer Lernrate von 0.0001 (Epochen=10) verläuft. Initialisieren Sie das Netz in Abschnitt 5 dafür NICHT!

Speichern Sie das akutelle Modell nachdem Sie Aufgabe 3 und 4 absolviert haben.

5. Starte die Vorhersage aus dem Live-Kamerabild. Lade zuvor das Modell, welches Sie nach Aufgabe 4 gespeichert haben. 

    Funktioniert die Live-Vorhersage wie von Ihnen erwartet? 
    Richten Sie die Kamera so aus, dass das Kamerabild den Trainingsbildern möglichst gut ähnelt und probieren Sie, ob die von Ihnen durchgeführten Gesten korrekt klassifiziert werden (Auch ein schwarzer Untergrund (bzw. ein farbiger Untergrund) funktionieren).